In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import transforms as transforms
from skimage import io
from skimage.transform import resize
from models import *


import os
import subprocess
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

%matplotlib inline


In [2]:
net = VGG('VGG19')
checkpoint = torch.load(os.path.join('FER2013_VGG19', 'PrivateTest_model.t7'))
net.load_state_dict(checkpoint['net'].state_dict())
# net.cuda()
net.eval()


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [3]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])


In [4]:
# imgclass_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
cut_size = 44
transform_test = transforms.Compose([
    transforms.TenCrop(cut_size),
    transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
])


In [5]:
def input2output(raw_img):
    gray = rgb2gray(raw_img)
    gray = resize(gray, (48,48), mode='symmetric').astype(np.uint8)
    img = gray[:, :, np.newaxis]
    img = np.concatenate((img, img, img), axis=2)
    img = Image.fromarray(img)
    inputs = transform_test(img)
    ncrops, c, h, w = np.shape(inputs)
    inputs = inputs.view(-1, c, h, w)
    inputs = Variable(inputs, volatile=True)
    outputs = net(inputs)
    outputs_avg = outputs.view(ncrops, -1).mean(0)
#     print(outputs)
    return outputs_avg


In [ ]:
# raw_img = io.imread('images/1.jpg')
# outputs_avg = input2output(raw_img)  # avg over crops
# score = F.softmax(outputs_avg)
# _, predicted = torch.max(outputs_avg.data, 0)
# float("{:.4f}".format(score[int(predicted)]))


In [6]:
os.chdir("../")
os.getcwd()

'/Users/kushalreddy_p/ASU Studies/ASU Work/CSI Lab/DataLogs/Log_Analyser'

In [7]:
# NON-AR Image Prediction

emotionCountData = []
df_EmotionCount = []
path = "../"
subpath = ""
os.chdir(path+"AR_Less Data/")
users = subprocess.check_output('ls').splitlines()
for user in users:
    if(user.decode("utf-8").startswith("User")):
        subpath = "{}/img/".format(user.decode("utf-8"))
        os.chdir(subpath)
        sections = subprocess.check_output('ls').splitlines()
        for section in sections:
            subpath = "{}".format(section.decode("utf-8"))
            os.chdir(subpath)
            images = subprocess.check_output('ls').splitlines()
            for imagename in images:
                imagename =imagename.decode("utf-8")
                raw_img = io.imread(imagename)
                outputs_avg = input2output(raw_img)
                score = F.softmax(outputs_avg)
                _, predicted = torch.max(outputs_avg.data, 0)
                max_index = int(predicted)
                emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
                predicted_emotion = emotions[max_index]
                em = [False, False, False, False,False, False,False ]
                em[max_index] = True
                emotionCountData = [user.decode("utf-8"), section.decode("utf-8"), imagename, predicted_emotion, em[0], em[1], em[2], em[3], em[4], em[5], em[6]]
                df_EmotionCount.append(emotionCountData)
                
            os.chdir("..")
        os.chdir("../..")
#                 gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#                 faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)
#                 for (x, y, w, h) in faces_detected:
#                     cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), thickness=7)
#                     roi_gray = gray_img[y:y + w, x:x + h]  # cropping region of interest i.e. face area from  image
#                     roi_gray = cv2.resize(roi_gray, (48, 48))
#                     img_pixels = image.img_to_array(roi_gray)
#                     img_pixels = np.expand_dims(img_pixels, axis=0)
#                     img_pixels /= 255
#                     predictions = xception.predict(img_pixels)
#                     max_index = np.argmax(predictions[0])
#                     emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
#                     predicted_emotion = emotions[max_index]
#                     em = [False, False, False, False,False, False,False ]
#                     em[max_index] = True
#                     emotionCountData = [user.decode("utf-8"), section.decode("utf-8"), imagename, predicted_emotion, em[0], em[1], em[2], em[3], em[4], em[5], em[6]]
#                     df_EmotionCount.append(emotionCountData)
#             os.chdir("..")
#         os.chdir("../..")
        
# Convert the list of records to DataFrame
df_EmotionCount_NAR = pd.DataFrame(df_EmotionCount, columns = ['User', 'Section', 'ImageName', 'Emotion', 'Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral' ])





User_1
Section_0


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  # Remove the CWD from sys.path while we load stuff.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Section_1


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  # Remove the CWD from sys.path while we load stuff.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Section_2


...

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  # Remove the CWD from sys.path while we load stuff.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Section_2


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  # Remove the CWD from sys.path while we load stuff.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Section_3


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  # Remove the CWD from sys.path while we load stuff.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [8]:
df_EmotionCount_NAR['Emotion'].value_counts()

happy      2080
neutral     109
Name: Emotion, dtype: int64

In [9]:
emotionCountData = []
df_EmotionCount = []
path = "../"
subpath = ""
os.chdir(path+"AR Data/")
users = subprocess.check_output('ls').splitlines()
for user in users:
    if(user.decode("utf-8").startswith("User")):
        subpath = "{}/img/".format(user.decode("utf-8"))
        os.chdir(subpath)
        sections = subprocess.check_output('ls').splitlines()
        for section in sections:
            subpath = "{}".format(section.decode("utf-8"))
            os.chdir(subpath)
            images = subprocess.check_output('ls').splitlines()
            for imagename in images:
                imagename =imagename.decode("utf-8")
                raw_img = io.imread(imagename)
                outputs_avg = input2output(raw_img)
                score = F.softmax(outputs_avg)
                _, predicted = torch.max(outputs_avg.data, 0)
                max_index = int(predicted)
                emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
                predicted_emotion = emotions[max_index]
                em = [False, False, False, False,False, False,False ]
                em[max_index] = True
                emotionCountData = [user.decode("utf-8"), section.decode("utf-8"), imagename, predicted_emotion, em[0], em[1], em[2], em[3], em[4], em[5], em[6]]
                df_EmotionCount.append(emotionCountData)
                
            os.chdir("..")
        os.chdir("../..")
        
# Convert the list of records to DataFrame
df_EmotionCount_AR = pd.DataFrame(df_EmotionCount, columns = ['User', 'Section', 'ImageName', 'Emotion', 'Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral' ])




/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  # Remove the CWD from sys.path while we load stuff.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  # Remove the CWD from sys.path while we load stuff.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/Library/Frameworks/Python.framework/Versions/

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  # Remove the CWD from sys.path while we load stuff.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  # Remove the CWD from sys.path while we load stuff.
/Library/Frameworks/Python.framework/Versions/

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  # Remove the CWD from sys.path while we load stuff.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  # Remove the CWD from sys.path while we load stuff.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/Library/Frameworks/Python.framework/Versions/

In [10]:
df_EmotionCount_AR['Emotion'].value_counts()

happy      6420
neutral     100
Name: Emotion, dtype: int64

In [17]:
checkpoint

{'net': VGG(
   (features): Sequential(
     (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (2): ReLU(inplace=True)
     (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (5): ReLU(inplace=True)
     (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
     (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (9): ReLU(inplace=True)
     (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (12): ReLU(inplace=True)
     (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  

In [ ]:
raw_img = io.imread('images/1.jpg')
outputs_avg = input2output(raw_img)  # avg over crops
score = F.softmax(outputs_avg)
_, predicted = torch.max(outputs_avg.data, 0)
float("{:.4f}".format(score[int(predicted)]))